<a href="https://colab.research.google.com/github/sg2e16/Freddie-Mac-Logistic-Regression-and-Scorecard/blob/master/Freddie_Mac_Logistic_Regression_and_Scorecard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf 'sample_data'

**Importing:**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip install scorecardpy
import scorecardpy as sc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
mdfm = pd.read_csv("/content/gdrive/My Drive/MergedDataFM/mergedataFM.csv")

**Key:**

**CREDIT SCORE -** A number, prepared by third parties, summarizing
the borrower’s creditworthiness, which may be indicative of the
likelihood that the borrower will timely repay future obligations.
Generally, the credit score disclosed is the score known at the time of
acquisition and is the score used to originate the mortgage.

**FIRST PAYMENT DATE -** The date of the first scheduled mortgage
payment due under the terms of the mortgage note.

**MATURITY DATE -** The month in which the final monthly payment on
the mortgage is scheduled to be made as stated on the original
mortgage note.

**MORTGAGE INSURANCE PERCENTAGE (MI %) -** The percentage
of loss coverage on the loan, at the time of Freddie Mac’s purchase of
the mortgage loan that a mortgage insurer is providing to cover losses
incurred as a result of a default on the loan. Only primary mortgage
insurance that is purchased by the Borrower, lender or Freddie Mac is
disclosed. Mortgage insurance that constitutes “credit enhancement”
that is not required by Freddie Mac’s Charter is not disclosed.
Amounts of mortgage insurance reported by Sellers that are less than
1% or greater than 55% will be disclosed as “Not Available,” which will
be indicated 999. No MI will be indicated by three zeros.

**ORIGINAL COMBINED LOAN-TO-VALUE (CLTV) –** In the case of a
purchase mortgage loan, the ratio is obtained by dividing the original
mortgage loan amount on the note date plus any secondary mortgage
loan amount disclosed by the Seller by the lesser of the mortgaged
property’s appraised value on the note date or its purchase price. In
the case of a refinance mortgage loan, the ratio is obtained by dividing
the original mortgage loan amount on the note date plus any
secondary mortgage loan amount disclosed by the Seller by the
mortgaged property’s appraised value on the note date. If the
secondary financing amount disclosed by the Seller includes a home
equity line of credit, then the CLTV calculation reflects the disbursed
amount at closing of the first lien mortgage loan, not the maximum
loan amount available under the home equity line of credit. In the
case of a seasoned mortgage loan, if the Seller cannot warrant that
the value of the mortgaged property has not declined since the note
date, Freddie Mac requires that the Seller must provide a new
appraisal value, which is used in the CLTV calculation. In certain
cases, where the Seller delivered a loan to Freddie Mac with a special
code indicating additional secondary mortgage loan amounts, those
amounts may have been included in the CLTV calculation.
If the LTV is < 80 or > 200 or Not Available, set the CLTV to ‘Not
Available.’ If the CLTV is < LTV, set the CLTV to ‘Not Available.’
This disclosure is subject to the widely varying standards originators
use to verify Borrowers’ secondary mortgage loan amounts and will
not be updated.

**ORIGINAL DEBT-TO-INCOME (DTI) RATIO -** Disclosure of the debt
to income ratio is based on (1) the sum of the borrower's monthly debt
payments, including monthly housing expenses that incorporate the
mortgage payment the borrower is making at the time of the delivery
of the mortgage loan to Freddie Mac, divided by (2) the total monthly
income used to underwrite the loan as of the date of the origination of
the such loan.
Ratios greater than 65% are indicated that data is Not Available. All
loans in the HARP dataset will be disclosed as Not Available.
This disclosure is subject to the widely varying standards originators
use to verify Borrowers’ assets and liabilities and will not be updated. 

**ORIGINAL UPB -** The UPB of the mortgage on the note date. UPB = Unpaid Principal Balance, this is the portion of the loan at a certain point in time that has not yet been remitted to the lender.Amount will be rounded to  the nearest $1,000.

**ORIGINAL LOAN-TO-VALUE (LTV) -** In the case of a purchase
mortgage loan, the ratio obtained by dividing the original mortgage
loan amount on the note date by the lesser of the mortgaged
property’s appraised value on the note date or its purchase price.
In the case of a refinance mortgage loan, the ratio obtained by dividing
the original mortgage loan amount on the note date and the
mortgaged property’s appraised value on the note date.
In the case of a seasoned mortgage loan, if the Seller cannot warrant
that the value of the mortgaged property has not declined since the
note date, Freddie Mac requires that the Seller must provide a new
appraisal value, which is used in the LTV calculation.

**ORIGINAL INTEREST RATE -** The original note rate as indicated on
the mortgage note.

**PREPAYMENT PENALTY MORTGAGE (PPM) FLAG -** Denotes
whether the mortgage is a PPM. A PPM is a mortgage with respect to
which the borrower is, or at any time has been, obligated to pay a
penalty in the event of certain repayments of principal.

Y = PPM
N = Not PPM

**PROPERTY STATE -** A two-letter abbreviation indicating the state or
territory within which the property securing the mortgage is located.

**PROPERTY TYPE -** Denotes whether the property type secured by
the mortgage is a condominium, leasehold, planned unit development
(PUD), cooperative share, manufactured home, or Single Family
home.
If the Property Type is Not Available, this will be indicated by 99. 

**LOAN SEQUENCE NUMBER -** Unique identifier assigned to each
loan. 

**ORIGINAL LOAN TERM -** A calculation of the number of scheduled
monthly payments of the mortgage based on the First Payment Date
and Maturity Date.

**SERVICER NAME -** The entity acting in its capacity as the servicer of
mortgages to Freddie Mac as of the last period for which loan activity
is reported in the Dataset.
Servicer Name will be disclosed for servicers with a total Original UPB
representing 1% or more of the total Original UPB of all loans in the
Dataset for a given calendar quarter. Otherwise, the Servicer Name
will be set to “Other Servicers”.

**Default -** Creating default value "1" or "0" on current loan deliquency status that are >= 6 and on zero balance code that = 2, 3, 9, 15.

In [0]:
indexNames = mdfm[mdfm['FirstPaymentDate'] >= 201500 ].index
mdfm.drop(indexNames , inplace=True)

In [0]:
mdfm.sort_values("FirstPaymentDate", inplace = True)
mdfm.head(449798)

**Exploratory Data Analysis:**

In [0]:
##Listing the columns
cols = list(mdfm.columns.values)
print(cols)

In [0]:
##Reordering the columns
mdfm = mdfm [['LoanSequenceNumber',         ## 1
              'FirstPaymentDate',           ## 2
              'MaturityDate',               ## 3
              'CreditScore',                ## 4
              'MortgageInsurancePercentage',## 5
              'OroginalCLTV',               ## 6
              'OriginalDTIRatio',           ## 7
              'OriginalUPB',                ## 8
              'OriginalLTV',                ## 9
              'OriginalInterestRate',       ## 10
              'OriginalLoanTerm',           ## 11
              'PPMFlag',                    ## 12
              'PropertyState',              ## 13
              'PropertyType',               ## 14
              'ServiceName',                ## 15              
              'Default']]                   ## 16
mdfm.head(10)

In [0]:
mdfm.shape

In [0]:
mdfm.info()

In [0]:
count_no_Def = len(mdfm[mdfm['Default']==0])
count_Def = len(mdfm[mdfm['Default']==1])
pct_of_no_Def = count_no_Def/(count_no_Def+count_Def)
print("Percentage of no Default is", pct_of_no_Def*100)
pct_of_Def = count_Def/(count_no_Def+count_Def)
print("Percentage of Default", pct_of_Def*100)

**Null Values identifacation and cleaning:**

If the Max Value Below  is equal to 9999, 999 or 99 then a Null Value is present.

In [0]:
mdfm.describe()

In [0]:
##Drops 63 Cases
indexNames = mdfm[mdfm['MortgageInsurancePercentage'] == 999 ].index
mdfm.drop(indexNames , inplace=True)

In [0]:
##Drops 18 Cases
indexNames = mdfm[mdfm['OroginalCLTV'] == 999 ].index
mdfm.drop(indexNames , inplace=True)

In [0]:
##Drops 14 Cases
indexNames = mdfm[mdfm['OriginalLTV'] == 999 ].index
mdfm.drop(indexNames , inplace=True)

In [0]:
##Drops 23 Cases
mdfm.loc[mdfm['PropertyType'] == '99' , 'PropertyType'] = np.nan
mdfm.dropna(inplace=True)
##mdfm['PropertyType'].value_counts(dropna=False)

In [0]:
##Edits 925 Cases of "CreditScore" to the mean
mdfm.loc[mdfm['CreditScore'] == 9999, 'CreditScore'] = np.nan
mdfm['CreditScore'].fillna((mdfm['CreditScore'].mean()), inplace=True)

In [0]:
##Edits 5032 Cases of "OriginalDTIRatio" to the mean
mdfm.loc[mdfm['OriginalDTIRatio'] == 999, 'OriginalDTIRatio'] = np.nan
mdfm['OriginalDTIRatio'].fillna((mdfm['OriginalDTIRatio'].mean()), inplace=True)

In [0]:
##mdfm.head()

In [0]:
##mdfm.describe()

**Identifying Outliers:**

In [0]:
##Creates the pair plot (hashed out for load times) 
##sns.pairplot(mdfm.iloc[:,3:11])

In [0]:
##box = mdfm.iloc[:,3:11].boxplot(grid=False, rot=45, fontsize=13)

In [0]:
##Cut below 550 !Unknown!
sns.distplot(mdfm['CreditScore'])

In [0]:
## !Unknown! Cut after 40
sns.distplot(mdfm['MortgageInsurancePercentage'])

In [0]:
##Cut above 124
sns.distplot(mdfm['OroginalCLTV'])

In [0]:
##Keep
sns.distplot(mdfm['OriginalDTIRatio'])

In [0]:
##Cut above 800000 !Unsure!
sns.distplot(mdfm['OriginalUPB'])

In [0]:
##Cut below 20
sns.distplot(mdfm['OriginalLTV'])

In [0]:
##Keep maybe cut above 10
sns.distplot(mdfm['OriginalInterestRate'])

In [0]:
##Cut above 500 below 120
sns.distplot(mdfm['OriginalLoanTerm'])

In [0]:
##mdfm = mdfm.loc[(mdfm['CreditScore'] >= 550) 
##                & (mdfm['MortgageInsurancePercentage'] <= 40) 
##                & (mdfm['OroginalCLTV'] <= 124)
##                & (mdfm['OriginalUPB'] <= 800000)
##                & (mdfm['OriginalLTV'] >= 20)
##                & (mdfm['OriginalInterestRate'] <= 10)
##                & (mdfm['OriginalLoanTerm'] <= 120)
##                & (mdfm['OriginalLoanTerm'] >= 500)]

In [0]:
##box = mdfm.iloc[:,3:11].boxplot(grid=False, rot=45, fontsize=13)

In [0]:
##mdfm.sort_values("FirstPaymentDate", inplace = True)
##mdfm.head(449798)

**Creating a Training and Test Datasets:**

In [0]:
train, test = sc.split_df(mdfm.iloc[:,3:],##:,3:16
                          y = 'Default',
                           ratio = 0.7, seed = 28845617).values()

**Checking for independence between features:**

In [0]:
mdfm.corr()

In [0]:
sns.heatmap(mdfm.corr())

In [0]:
train = train.drop(['OroginalCLTV'],axis = 1)
test = test.drop(['OroginalCLTV'],axis = 1)

In [0]:
##trainND = train.copy()
##trainND.drop(['Default'],axis=1,inplace=True)

In [0]:
##PPMFlag = pd.get_dummies(trainND['PPMFlag'],drop_first=True)
##PropertyState = pd.get_dummies(trainND['PropertyState'],drop_first=True)
##PropertyType = pd.get_dummies(trainND['PropertyType'],drop_first=True)
##ServiceName = pd.get_dummies(trainND['ServiceName'],drop_first=True)

##trainND.drop(['PPMFlag','PropertyState',
##           'PropertyType','ServiceName'],axis=1,inplace=True)
##trainND = pd.concat([trainND,PPMFlag,PropertyState,PropertyType,ServiceName],axis=1)

In [0]:
##import statsmodels.api as sm
##logit_model=sm.Logit(train['Default'],trainND)
##result=logit_model.fit(method = 'powell')##method='bfgs'
##print(result.summary2())

**Coding the Nominal Variables using Weights of Evidence (WoE):**

In [0]:
##Option "1"

bins = sc.woebin(train, y = 'Default', 
                 min_perc_fine_bin=0.02, # How many bins to cut initially into
                 min_perc_coarse_bin=0.05,  # Minimum percentage per final bin
                 stop_limit=0.1, # Minimum information value 
                 max_num_bin=8, # Maximum number of bins
                 method='tree')

In [0]:
sc.woebin_plot(bins)

In [0]:
##breaks_adj = sc.woebin_adj(train, "Default", bins)

In [0]:
##The breaks for woe as found by the Manual Bin
breaks_adj = {'CreditScore': [690,730,770,790],
              'FirstPaymentDate': [200300,200500], ## New
              'MaturityDate': [203400.0,203600.0], ## New
              'MortgageInsurancePercentage': [8.0,30.0],
              'OriginalDTIRatio':[30.0,37.0,46.0,49.0],
              'OriginalInterestRate':[4.0,5.0],
              'OriginalLTV':[40.0,58.0,82.0,92.0],
              'OriginalLoanTerm':[200.0, 300.0 ],##250.0
              'OriginalUPB':[70000.0,170000.0,260000.0,400000.0],
              'OroginalCLTV':[40.0,58.0,82.0],
              'PPMFlag':['N%,%Y'],
              'PropertyState':['AK%,%VI%,%GU%,%ND%,%VA%,%KS%,%NE%,%MN%,%CO%,%MO%,%SD%,%WY%,%VT%,%TX%,%TN%,%NC%,%KY%,%WI%,%AR%,%CA%,%NH','DE%,%AL%,%MA%,%IA%,%WA%,%WV%,%MI%,%OR%,%IN%,%UT%,%PA%,%RI%,%OK%,%CT%,%MT%,%IL%,%OH%,%GA%,%MD%,%AZ%,%ID%,%MS%,%LA','ME%,%DC%,%SC%,%NM%,%HI%,%NY%,%NJ%,%NV','PR%,%FL'],
              'PropertyType': ['99%,%CP%,%PU','SF','CO%,%MH'],
              'ServiceName': ['LAKEVIEWLOANSERVICIN%,%HOMEPOINTFINANCIALCO%,%IMPACMTGECORP%,%LOANDEPOTCOM_LLC%,%MATRIXFINANCIALSERVI%,%FREMONTBANK%,%FREEDOMMTGECORP%,%FRANKLINAMERICANMTGE%,%NEWRESIDENTIALMTGELL%,%FIRSTNATIONWIDEMTGEC%,%PAMMSRTRUST1_LLC%,%COLORADOFEDSAVINGSBA%,%HOMESIDELENDING_INC%,%PHHMTGECORP%,%COLONIALSAVINGS_FA%,%REGIONSBANKDBAREGION%,%SENECAMTGESERVICING_%,%CENDANTMTGECORPORATI%,%SPECIALIZEDLOANSERVI%,%STEARNSLENDING_LLC%,%UNIONSAVINGSBANK%,%ARVESTCENTRALMTGECO%,%AMTRUSTBANK%,%AMERIHOMEMTGECOMPANY%,%WASHINGTONMUTUALHOME%,%PODIUMMTGECAPITALLLC%,%HSBCMTGECORPUSA%,%QUICKENLOANINC','PENNYMACCORP%,%FIFTHTHIRDMTGECO%,%SUNTRUSTMORTGAGE_INC%,%WELLSFARGOHOMEMORTGA%,%PROVIDENTFUNDINGASSO%,%ROUNDPOINTMTGESERVIC%,%PINGORALOANSERVICING%,%CALIBERHOMELOANS_INC%,%PRINCIPALRESIDENTIAL','BRANCHBANKING&TRUSTC%,%OLDKENTMTGECO%,%ALLYBANK%,%FTMTGESERVICES_INC%,%COUNTRYWIDE%,%PNCBANK_NATL%,%WASHINGTONMUTUALBANK%,%CHASEMANHATTANMTGECO%,%USBANKNA%,%CHASEHOMEFINANCELLC%,%GMACMTGECORP%,%CHASEMTGECO%,%NATLCITYMTGECO','FIFTHTHIRDBANK%,%WELLSFARGOBANK_NA%,%TAYLOR_BEAN&WHITAKER%,%RUSHMORELOANMANAGEME%,%GMACMORTGAGE_LLC%,%ABNAMROMTGEGROUP_INC%,%PNCMTGESERVICES_INC%,%Other servicers%,%METLIFEHOMELOANS_ADI%,%CENTRALMTGECO%,%BANKOFAMERICA_NA','JPMORGANCHASEBANK_NA%,%CITIMORTGAGE_INC%,%NATIONSTARMTGELLCDBA%,%FLAGSTARCAPITALMARKE%,%OCWENLOANSERVICING_L%,%BACHOMELOANSERVICING%,%NATIONSTARMTGELLC' ]}

In [0]:
##Option "1"

bins_adj = sc.woebin(train, y="Default", breaks_list=breaks_adj)
train_woe = sc.woebin_ply(train, bins_adj)
test_woe = sc.woebin_ply(test, bins_adj)

In [0]:
##train_woe.describe()

In [0]:
##train_woe.head()

**IV Values Key:**
- $IV < 0.02$: Remove.
- $0.02 \le IV < 0.1$: Remove.
- $0.1 \le IV < 0.3$: Keep.
- $0.3 \le IV < 1$: Keep.
- $1 \le IV $: Keep/Review. Study if error in calculation (i.e. WoE leaves a category with 100% goods or bads) or if variable is capturing future information.

In [0]:
sc.iv(train_woe, 'Default')

In [0]:
##Without Dropping Higher ROC 85% over 83%
##Dropping IVs below 0.1
##train_woe = train_woe.drop(['MortgageInsurancePercentage_woe',
##                            'PropertyType_woe','OriginalUPB_woe',]
##                           ,axis = 1)
##test_woe = test_woe.drop(['MortgageInsurancePercentage_woe',
##                          'PropertyType_woe','OriginalUPB_woe',]
##                           ,axis = 1)
##train_woe.head()

In [0]:
##test_woe.head()

**Logistic Regression:**

In [0]:
trainLog = train.copy()
trainLog.shape

In [0]:
testLog = test.copy()
testLog.shape

In [0]:
trainLogWoe = train_woe.copy()
trainLogWoe.shape

In [0]:
testLogWoe = test_woe.copy()
testLogWoe.shape

In [0]:
mdfm_logregA = LogisticRegression(penalty='l1', # Type of penalization l1 = lasso, l2 = ridge
                                     tol=0.0001, # Tolerance for parameters
                                     C=1.0, # Penalty constant, see below
                                     fit_intercept=True, # Use constant?
                                     class_weight='balanced', # Weights, see below
                                     random_state=28845617, # Random seed
                                     max_iter=100, # Maximum iterations
                                     verbose=1, # Show process. 1 is yes.
                                     solver = 'saga',
                                     warm_start=False) # Train anew or start from previous weights. For repeated training. 

In [0]:
##trainLogWoe.isna().sum()

In [0]:
mdfm_logregA.fit(X = trainLogWoe.iloc[:,1:],
                    y = trainLogWoe['Default'] # The target
                   )

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 16 seconds


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.9s finished


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1',
          random_state=28845617, solver='saga', tol=0.0001, verbose=1,
          warm_start=False)

In [0]:
coef_df = pd.concat([pd.DataFrame({'column': trainLogWoe.columns[1:]}), 
                    pd.DataFrame(np.transpose(mdfm_logregA.coef_))],
                    axis = 1)
coef_df

In [0]:
mdfm_logregA.intercept_

In [0]:
pred_class_train_log = mdfm_logregA.predict(trainLogWoe.iloc[:, 1:])
probs_train_log = mdfm_logregA.predict_proba(trainLogWoe.iloc[:, 1:])
print(probs_train_log, pred_class_train_log)

In [0]:
pred_class_test_log = mdfm_logregA.predict(testLogWoe.iloc[:, 1:])
probs_test_log = mdfm_logregA.predict_proba(testLogWoe.iloc[:, 1:])
print(probs_test_log, pred_class_test_log)

In [0]:
##confusion_matrix(y_true = testLogWoe['Default'], y_pred = pred_class_test_log)

In [0]:
##Creating the confusion matrix for Logistic Regression
##confusion_matrix_rf = confusion_matrix(y_true = testLogWoe['Default'], 
##                    y_pred = pred_class_test_log)
##confusion_matrix_rf = confusion_matrix_rf.astype('float') / confusion_matrix_rf.sum(axis=1)[:, np.newaxis]
##df_cm = pd.DataFrame(
##        confusion_matrix_rf, index=['Safe', 'Default'], columns=['Safe', 'Default'],)
##figsize = (10,7)
##fontsize=14
##fig = plt.figure(figsize=figsize)
##heatmap = sns.heatmap(df_cm, annot=True, fmt='.2f')
##heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, 
##                             ha='right', fontsize=fontsize)
##heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45,
##                             ha='right', fontsize=fontsize)
##plt.ylabel('True label')
##plt.xlabel('Predicted label')
##plt.show()

In [0]:
roc_auc_score(y_true = trainLogWoe['Default'], y_score = probs_train_log[:,1])

In [0]:
# Calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(trainLogWoe['Default'], probs_train_log[:,1])

# Save the AUC in a variable to display it. Round it first
auc = np.round(roc_auc_score(y_true = testLogWoe['Default'], 
                             y_score = probs_test_log[:,1]),
              decimals = 3)

# Create and show the plot
plt.plot(fpr,tpr,label="MDFM Train, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [0]:
roc_auc_score(y_true = testLogWoe['Default'], y_score = probs_test_log[:,1])

In [0]:
# Calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(testLogWoe['Default'], probs_test_log[:,1])

# Save the AUC in a variable to display it. Round it first
auc = np.round(roc_auc_score(y_true = testLogWoe['Default'], 
                             y_score = probs_test_log[:,1]),
              decimals = 3)

# Create and show the plot
plt.plot(fpr,tpr,label="MDFM Test, auc="+str(auc))
plt.legend(loc=4)
plt.show()

**Scorecard:**

In [0]:
mdfm_sc = sc.scorecard(bins_adj, mdfm_logregA, 
             trainLogWoe.columns[1:], # The column names in the trained LR
             points0=1000, # Base points
             odds0=0.01, # Base odds
             pdo=50) # PDO 

In [0]:
mdfm_sc

In [0]:
train_score = sc.scorecard_ply(trainLog, mdfm_sc, 
                               print_step=0)
test_score = sc.scorecard_ply(testLog, mdfm_sc, 
                               print_step=0)

In [0]:
train_score.describe()